# 🚗 Sprint Challenge 4 – Previsão de Acidentes com LSTMs
## Case Sompo: Antecipando Padrões de Risco em Rodovias Brasileiras

---

**Equipe Big 5**
- Lucca Phelipe Masini - RM 564121
- Luiz Henrique Poss - RM 562177
- Luis Fernando de Oliveira Salgado - RM 561401
- Igor Paixão Sarak - RM 563726
- Bernardo Braga Perobeli - RM 562468

---

## 🎯 Target Escolhido: Classificação Binária de Risco

- **Classe 0 - BAIXO/MÉDIO RISCO**: < 30% de acidentes severos
- **Classe 1 - ALTO RISCO**: ≥ 30% de acidentes severos

### Justificativa

Utilizamos **classificação binária** para identificar semanas de alto risco em rodovias. O threshold de 30% separa períodos normais de períodos críticos que requerem atenção especial. Esta abordagem:

- ✅ É mais robusta com features temporais limitadas
- ✅ Facilita tomada de decisão (alerta sim/não)
- ✅ Permite ao modelo aprender padrões distintos
- ✅ Alinha-se com necessidades práticas de gestão de risco

---

## 📚 Passo 1: Instalação e Importação de Bibliotecas

Primeiro, vamos instalar e importar todas as bibliotecas necessárias para o projeto.


In [ ]:
!pip install openpyxl --quiet
print("✅ Bibliotecas instaladas!")

---

## 📥 Passo 2: Carregamento dos Dados

Carregamos os dados diretamente do GitHub para garantir reprodutibilidade total.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import urllib.request
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("📚 Bibliotecas importadas!")

---

## 🎯 Passo 3: Pré-processamento e Criação da Variável Target

Criamos a variável binária `severo` que identifica acidentes com mortos ou feridos graves.


In [ ]:
print("📥 Baixando dataset da PRF do GitHub...")

github_raw_url = 'https://raw.githubusercontent.com/9luis7/lstm-acidentes-prf/main/dados/datatran2025.xlsx'
output_filename = 'dados_acidentes.xlsx'

try:
    urllib.request.urlretrieve(github_raw_url, output_filename)
    df = pd.read_excel(output_filename)
    print(f"✅ Dataset carregado: {len(df):,} acidentes")
    print("\n📊 Período:", df['data_inversa'].min(), "até", df['data_inversa'].max())
    print("📊 Estados:", df['uf'].nunique(), "UFs")
except Exception as e:
    print(f"❌ Erro: {e}")
    raise

---

## 🔄 Passo 4: Agregação Semanal

Transformamos acidentes individuais em séries temporais semanais por estado.


In [ ]:
print("🎯 Criando variável target 'severo'...")

df['horario'] = pd.to_datetime(df['horario'], format='%H:%M:%S', errors='coerce').dt.time
df['severo'] = ((df['mortos'] > 0) | (df['feridos_graves'] > 0)).astype(int)

colunas_relevantes = ['data_inversa', 'horario', 'uf', 'br', 'km', 'pessoas', 'veiculos', 'severo']
df_limpo = df[colunas_relevantes].copy()
df_limpo['horario'].fillna(pd.to_datetime('12:00:00').time(), inplace=True)

print("✅ Variável 'severo' criada!")
print("\n📊 Distribuição:")
print(df_limpo['severo'].value_counts(normalize=True))

---

## 🎨 Passo 5: Feature Engineering

Criamos 12 features enriquecidas: temporais, sazonalidade e histórico (lags).


In [ ]:
print("🔄 Agregando dados em séries temporais semanais...")

df_indexed = df_limpo.set_index('data_inversa')

weekly_df = df_indexed.groupby([pd.Grouper(freq='W'), 'uf']).agg(
    total_acidentes=('severo', 'count'),
    acidentes_severos=('severo', 'sum'),
    pessoas_total=('pessoas', 'sum'),
    veiculos_total=('veiculos', 'sum'),
    pessoas_media=('pessoas', 'mean'),
    veiculos_media=('veiculos', 'mean')
).reset_index()

weekly_df['prop_severos'] = np.where(
    weekly_df['total_acidentes'] > 0,
    weekly_df['acidentes_severos'] / weekly_df['total_acidentes'],
    0
)

print(f"✅ Dados agregados: {len(weekly_df):,} semanas × estados")

---

## 🔢 Passo 6: Criação de Sequências Temporais

Criamos janelas de 8 semanas para prever a semana seguinte. Normalizamos os dados com MinMaxScaler.


In [ ]:
print("🎨 Criando features temporais e de histórico...")

# Temporais
weekly_df['dia_semana'] = weekly_df['data_inversa'].dt.dayofweek
weekly_df['mes'] = weekly_df['data_inversa'].dt.month
weekly_df['fim_semana'] = weekly_df['dia_semana'].isin([5, 6]).astype(int)

# Sazonalidade
weekly_df['sazonalidade_sen'] = np.sin(2 * np.pi * weekly_df['data_inversa'].dt.dayofyear / 365)
weekly_df['sazonalidade_cos'] = np.cos(2 * np.pi * weekly_df['data_inversa'].dt.dayofyear / 365)

# Lags
for lag in [1, 2, 3]:
    weekly_df[f'prop_severos_lag{lag}'] = weekly_df.groupby('uf')['prop_severos'].shift(lag)

# Estatísticas
weekly_df['prop_severos_ma3'] = weekly_df.groupby('uf')['prop_severos'].rolling(3).mean().reset_index(0, drop=True)
weekly_df['prop_severos_tendencia'] = weekly_df.groupby('uf')['prop_severos'].diff()
weekly_df['prop_severos_volatilidade'] = weekly_df.groupby('uf')['prop_severos'].rolling(3).std().reset_index(0, drop=True)

print("✅ Features criadas!")
print(f"   Total: {len(weekly_df.columns)} colunas")

---

## 🎯 Passo 7: Transformação para Classificação

Como regressão falhou (R² negativo), transformamos o problema em classificação de 4 níveis de risco.


---

## 🏗️ Passo 8: Construção do Modelo LSTM (SIMPLIFICADO)

**Arquitetura Simplificada** para evitar overfitting:
- Uma camada LSTM (32 neurônios)
- Dropout aumentado (0.3)
- Learning rate reduzido (0.0005)
- Early stopping mais restritivo (patience=10)

**Class Weights** para corrigir desbalanceamento entre classes.


In [ ]:
from sklearn.preprocessing import MinMaxScaler

print("🔢 Preparando sequências para LSTM...")

features_colunas = [
    'prop_severos', 'pessoas_media', 'veiculos_media', 'fim_semana',
    'sazonalidade_sen', 'sazonalidade_cos',
    'prop_severos_lag1', 'prop_severos_lag2', 'prop_severos_lag3',
    'prop_severos_ma3', 'prop_severos_tendencia', 'prop_severos_volatilidade'
]

df_features = weekly_df.set_index('data_inversa').sort_index()
df_features = df_features[features_colunas].copy()
df_features = df_features.dropna()

# IMPORTANTE: Criar CLASSES ANTES da normalização
print("\n📊 Criando classes BINARIAS ANTES da normalização...")
target_values = df_features['prop_severos'].values

def criar_classes_risco(y_data):
    """Classificação binária: 0 = Baixo/Médio (< 30%), 1 = Alto Risco (>= 30%)"""
    classes = np.zeros_like(y_data, dtype=int)
    classes[y_data >= 0.30] = 1  # ALTO RISCO
    return classes

# Criar classes dos valores originais
classes_originais = criar_classes_risco(target_values)

print("\n📊 Distribuição das classes (ANTES da normalização):")
unique, counts = np.unique(classes_originais, return_counts=True)
nomes_classes = ['BAIXO/MEDIO (<0.30)', 'ALTO RISCO (>=0.30)']
for u, c in zip(unique, counts):
    print(f"   Classe {u} - {nomes_classes[u]}: {c:4d} ({c/len(classes_originais)*100:5.1f}%)")

# Separar features de target
features_sem_target = [col for col in features_colunas if col != 'prop_severos']
target_col = 'prop_severos'

# Normalizar apenas as FEATURES (sem o target)
df_features_input = df_features[features_sem_target].copy()
scaler_features = MinMaxScaler(feature_range=(0, 1))
features_scaled = scaler_features.fit_transform(df_features_input.values)

# Usar o TARGET sem normalização (mantém valores originais)
target_values = df_features[target_col].values

n_passos_para_tras = 8
n_features = len(features_sem_target)

X, y = [], []
for i in range(n_passos_para_tras, len(features_scaled)):
    X.append(features_scaled[i-n_passos_para_tras:i, :])
    y.append(target_values[i])  # VALORES ORIGINAIS, não normalizados

X, y = np.array(X), np.array(y)

print(f"\n✅ Sequências criadas!")
print(f"   Shape X: {X.shape}")
print(f"   Shape y: {y.shape}")
print(f"   Range de y: [{y.min():.3f}, {y.max():.3f}]")

In [ ]:
from tensorflow.keras.utils import to_categorical

print("🎯 Transformando para CLASSIFICAÇÃO BINÁRIA...")

# Usar a função já definida anteriormente
def criar_classes_risco(y_data):
    """Classificação binária: 0 = Baixo/Médio (< 30%), 1 = Alto Risco (>= 30%)"""
    classes = np.zeros_like(y_data, dtype=int)
    classes[y_data >= 0.30] = 1  # ALTO RISCO
    return classes

# Dividir dados
split_index = int(len(X) * 0.85)
X_train, X_val = X[:split_index], X[split_index:]
y_train, y_val = y[:split_index], y[split_index:]

print(f"\n📊 Estatísticas de y_train:")
print(f"   Min: {y_train.min():.3f}, Max: {y_train.max():.3f}")
print(f"   Mean: {y_train.mean():.3f}, Std: {y_train.std():.3f}")

print(f"\n📊 Estatísticas de y_val:")
print(f"   Min: {y_val.min():.3f}, Max: {y_val.max():.3f}")
print(f"   Mean: {y_val.mean():.3f}, Std: {y_val.std():.3f}")

# Criar classes
y_train_classes = criar_classes_risco(y_train)
y_val_classes = criar_classes_risco(y_val)

# One-hot encoding (2 classes)
y_train_categorical = to_categorical(y_train_classes, num_classes=2)
y_val_categorical = to_categorical(y_val_classes, num_classes=2)

nomes_classes = ['BAIXO/MEDIO (<0.30)', 'ALTO RISCO (>=0.30)']

print("\n📊 Distribuição no TREINO:")
for i in range(2):
    count = (y_train_classes == i).sum()
    percent = count / len(y_train_classes) * 100
    print(f"   Classe {i} - {nomes_classes[i]}: {count:4d} ({percent:5.1f}%)")

print("\n📊 Distribuição na VALIDAÇÃO:")
for i in range(2):
    count = (y_val_classes == i).sum()
    percent = count / len(y_val_classes) * 100
    print(f"   Classe {i} - {nomes_classes[i]}: {count:4d} ({percent:5.1f}%)")

print("\n✅ Dados preparados!")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

print("🏗️  Construindo modelo LSTM SIMPLIFICADO...")

# MODELO SIMPLIFICADO PARA CLASSIFICAÇÃO BINÁRIA
model = Sequential([
    LSTM(units=64, return_sequences=False, input_shape=(n_passos_para_tras, n_features)),
    Dropout(0.3),
    Dense(units=16, activation='relu'),
    Dropout(0.3),
    Dense(units=2, activation='softmax')  # 2 classes
])

# LEARNING RATE
optimizer = Adam(learning_rate=0.001)
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("✅ Modelo construído!")
print("   Arquitetura: LSTM 64 → Dense 16 → Softmax 2")
print("   Dropout: 0.3")
print("   Learning rate: 0.001")
print("   SEM class weights - aprendizado natural")
model.summary()

# CALLBACKS
early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=0.00001, verbose=1)

print("\n✅ Callbacks configurados!")
print("   Early Stopping: patience=15")
print("   Reduce LR: patience=7")

---

## 🚀 Passo 9: Treinamento do Modelo

Treinamos o modelo com class weights para corrigir desbalanceamento. Monitoramos val_loss para evitar overfitting.


In [ ]:
print("\n" + "="*70)
print("🚀 TREINANDO MODELO LSTM")
print("="*70)
print("\n📚 Aprendizado natural SEM class weights")
print("⏱️  Aguarde 10-20 minutos...\n")

history = model.fit(
    X_train, y_train_categorical,
    epochs=100,
    batch_size=16,
    validation_data=(X_val, y_val_categorical),
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

print("\n" + "="*70)
print("✅ TREINAMENTO CONCLUÍDO!")
print("="*70)

---

## 🔧 Abordagem de Modelagem

### Decisão: Classificação Binária

Optamos por **classificação binária** em vez de 4 classes para:

1. **Maior robustez** - Modelo mais simples aprende melhor com features temporais limitadas
2. **Threshold claro** - 30% de acidentes severos separa risco normal de crítico
3. **Aplicação prática** - Alertas binários (sim/não) facilitam tomada de decisão
4. **Melhor convergência** - Modelo consegue distinguir 2 padrões distintos

### Técnicas Aplicadas

1. **Classes criadas ANTES da normalização** - Usar valores originais de `prop_severos`
2. **Normalizar apenas features** - Target mantém valores originais
3. **Arquitetura simplificada** - LSTM única (64 neurônios) + Dense (16)
4. **Aprendizado natural** - SEM class weights para evitar viés artificial

---



---

## 📊 Passo 10: Avaliação e Métricas

Avaliamos o modelo com accuracy, precision, recall e F1-score. Também geramos matriz de confusão.


---

## 📈 Passo 11: Visualizações

Geramos 6 gráficos: curvas de aprendizagem, matriz de confusão, comparação temporal, distribuição de probabilidades e acurácia por classe.


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

print("📊 Avaliando modelo...")

y_pred_proba = model.predict(X_val, verbose=0)
y_pred_classes = np.argmax(y_pred_proba, axis=1)

accuracy = accuracy_score(y_val_classes, y_pred_classes)

print("\n" + "="*70)
print("🎯 RESULTADOS FINAIS")
print("="*70)
print(f"\n🏆 ACURÁCIA: {accuracy:.2%}\n")

baseline_random = 0.25
baseline_majority = np.bincount(y_val_classes).max() / len(y_val_classes)

print("📊 Comparação com Baselines:")
print(f"   Random Guess: {baseline_random:.1%}")
print(f"   Classe mais comum: {baseline_majority:.1%}")
print(f"   Nosso modelo: {accuracy:.1%} ✅")

print("\n" + "="*70)
print("📊 RELATÓRIO POR CLASSE")
print("="*70 + "\n")
print(classification_report(y_val_classes, y_pred_classes, target_names=nomes_classes, digits=3))

---

## 💾 Passo 12: Salvamento do Modelo

Salvamos o modelo treinado no formato `.keras` para uso futuro.


In [ ]:
plt.figure(figsize=(16, 12))

# 1. Loss
plt.subplot(3, 2, 1)
plt.plot(history.history['loss'], label='Treino', color='blue', linewidth=2)
plt.plot(history.history['val_loss'], label='Validação', color='red', linewidth=2)
plt.title('Curvas de Aprendizagem - Loss', fontsize=14, fontweight='bold')
plt.xlabel('Épocas')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# 2. Accuracy
plt.subplot(3, 2, 2)
plt.plot(history.history['accuracy'], label='Treino', color='blue', linewidth=2)
plt.plot(history.history['val_accuracy'], label='Validação', color='red', linewidth=2)
plt.title('Curvas de Aprendizagem - Acurácia', fontsize=14, fontweight='bold')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend()
plt.grid(True, alpha=0.3)

# 3. Matriz de Confusão
plt.subplot(3, 2, 3)
cm = confusion_matrix(y_val_classes, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar_kws={'label': 'Contagem'})
plt.title('Matriz de Confusão', fontsize=14, fontweight='bold')
plt.ylabel('Real')
plt.xlabel('Previsto')
plt.xticks([0.5, 1.5], ['Baixo/Médio', 'Alto Risco'], rotation=0)
plt.yticks([0.5, 1.5], ['Baixo/Médio', 'Alto Risco'], rotation=0)

# 4. Comparação Temporal
plt.subplot(3, 2, 4)
plt.plot(y_val_classes, label='Real', marker='o', linewidth=2, markersize=5, alpha=0.7)
plt.plot(y_pred_classes, label='Previsto', marker='x', linestyle='--', linewidth=2, markersize=5, alpha=0.7)
plt.title('Comparação Temporal', fontsize=14, fontweight='bold')
plt.xlabel('Amostras')
plt.ylabel('Classe')
plt.yticks([0, 1], ['Baixo/Médio', 'Alto Risco'])
plt.legend()
plt.grid(True, alpha=0.3)

# 5. Distribuição de Probabilidades
plt.subplot(3, 2, 5)
labels_proba = ['Baixo/Médio', 'Alto Risco']
for i in range(2):
    plt.hist(y_pred_proba[:, i], bins=20, alpha=0.6, label=labels_proba[i], edgecolor='black')
plt.title('Distribuição de Probabilidades', fontsize=14, fontweight='bold')
plt.xlabel('Probabilidade')
plt.ylabel('Frequência')
plt.legend()
plt.grid(True, alpha=0.3)

# 6. Acurácia por Classe
plt.subplot(3, 2, 6)
acertos_por_classe = []
labels_acc = ['Baixo/Médio', 'Alto Risco']
for i in range(2):
    mask = (y_val_classes == i)
    if mask.sum() > 0:
        acertos = (y_pred_classes[mask] == i).sum() / mask.sum() * 100
        acertos_por_classe.append(acertos)
    else:
        acertos_por_classe.append(0)

colors = ['green' if acc > 50 else 'orange' if acc > 30 else 'red' for acc in acertos_por_classe]
bars = plt.bar(labels_acc, acertos_por_classe, color=colors, edgecolor='black')
plt.title('Acurácia por Classe', fontsize=14, fontweight='bold')
plt.ylabel('Acurácia (%)')
plt.ylim(0, 100)
plt.grid(True, alpha=0.3, axis='y')

for bar, acc in zip(bars, acertos_por_classe):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height, f'{acc:.1f}%', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✅ Visualizações geradas!")

In [ ]:
model_filename = 'modelo_lstm_classificacao_risco.keras'
model.save(model_filename)
print(f"💾 Modelo salvo: '{model_filename}'")
print("\n✅ Projeto concluído!")